In [22]:
import pandas as pd
import numpy as np

from stable_baselines import A2C, DQN
from stable_baselines.common.policies import *
# MlpPolicy
# LnMlpPolicy
# CnnPolicy
# LnCnnPolicy
from stable_baselines.common.vec_env import DummyVecEnv

import sys, os
sys.path.append('/home/jovyan/work/biddingStrategy/src')

from environments import SingleAgentTrainingEnv
from agents import UniformRandomAgent, GymRLAgent
from info_settings import OfferInformationSetting
from engine import MarketEngine

In [2]:
from agents import UniformRandomAgent, GymRLAgent

fixed_agents = [
    UniformRandomAgent('seller', 3, name='CHAM1-PV'),
    UniformRandomAgent('seller', 3, name='CHAM2-PV'),
    UniformRandomAgent('seller', 3, name='CHAM3-PV'),
    UniformRandomAgent('seller', 3, name='CHAM4-PV'),
    UniformRandomAgent('seller', 3, name='CHAM5-PV'),
    UniformRandomAgent('buyer', 5, name='CHAM1'),
    UniformRandomAgent('buyer', 5, name='CHAM2'),
    UniformRandomAgent('buyer', 5, name='CHAM3'),
    UniformRandomAgent('buyer', 5, name='CHAM4'),
]

rl_agent = GymRLAgent('buyer', 5, discretization=20,name='CHAM5')
setting = OfferInformationSetting(5)

def get_env(rl_agent, fixed_agents, setting):
    return SingleAgentTrainingEnv(rl_agent, fixed_agents, setting)

env = DummyVecEnv([lambda: get_env(rl_agent, fixed_agents, setting)]) # wrap it for baselines

In [3]:
model = DQN("LnMlpPolicy", env, verbose=1, learning_rate=0.05)
# MlpPolicy
# LnMlpPolicy
# CnnPolicy
# LnCnnPolicy







Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [4]:
%%time
model.learn(total_timesteps=10000)
#Timestep which is used to train agent use the historical data at only one point of time or randomly pick multiple point of time?
#engine use avg sold or avg bought comparing to bidPrice of historical data to determine rewards

--------------------------------------
| % time spent exploring  | 90       |
| episodes                | 100      |
| mean 100 episode reward | 0.3      |
| steps                   | 98       |
--------------------------------------
--------------------------------------
| % time spent exploring  | 80       |
| episodes                | 200      |
| mean 100 episode reward | 0.3      |
| steps                   | 198      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 70       |
| episodes                | 300      |
| mean 100 episode reward | 0.2      |
| steps                   | 298      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 60       |
| episodes                | 400      |
| mean 100 episode reward | 0.1      |
| steps                   | 398      |
--------------------------------------
--------------------------------------
| % time spent exploring 

--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 3700     |
| mean 100 episode reward | 0.4      |
| steps                   | 3698     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 3800     |
| mean 100 episode reward | 0.5      |
| steps                   | 3798     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 3900     |
| mean 100 episode reward | 0.3      |
| steps                   | 3898     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 4000     |
| mean 100 episode reward | 0.4      |
| steps                   | 3998     |
--------------------------------------
--------------------------------------
| % time spent exploring 

--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 7300     |
| mean 100 episode reward | 0.4      |
| steps                   | 7298     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 7400     |
| mean 100 episode reward | 0.4      |
| steps                   | 7398     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 7500     |
| mean 100 episode reward | 0.4      |
| steps                   | 7498     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 7600     |
| mean 100 episode reward | 0.4      |
| steps                   | 7598     |
--------------------------------------
--------------------------------------
| % time spent exploring 

In [5]:
rl_agent.model = model

In [6]:
def get_reward(agent, deals):
    if not agent.name in deals:
        return [0,0,0]
    deal_price = deals[agent.name]
    if(deal_price==0): reward=0
    if agent.role == 'buyer':
        if(deal_price!=0): reward = agent.reservation_price - deal_price
        return [reward,deal_price,agent.reservation_price]
#         return reward
    else:
        if(deal_price!=0):reward = deal_price - agent.reservation_price
        return [reward,deal_price,agent.reservation_price]
#         return reward 

In [17]:
def play_games(agents, setting, n_games=100, max_steps=30):
    buyer_ids =  [
        agent.name
        for agent in agents
        if agent.role == 'buyer'
    ]
    seller_ids =  [
        agent.name
        for agent in agents
        if agent.role == 'seller'
    ]
    buyer_ids_deal =  [
        agent.name+"_deal"
        for agent in agents
        if agent.role == 'buyer'
    ]
    seller_ids_deal =  [
        agent.name+"_deal"
        for agent in agents
        if agent.role == 'seller'
    ]
    buyer_ids_resev =  [
        agent.name+"_resev"
        for agent in agents
        if agent.role == 'buyer'
    ]
    seller_ids_resev =  [
        agent.name+"_resev"
        for agent in agents
        if agent.role == 'seller'
    ]
    ids = set(buyer_ids+ seller_ids)
    ids_info=set(buyer_ids_deal + seller_ids_deal+ buyer_ids_resev + seller_ids_resev)
    market = MarketEngine(buyer_ids, seller_ids, max_steps=max_steps)
    
    rewards = pd.DataFrame(0, index=np.arange(n_games), columns=ids)
#     rewards = pd.DataFrame(0, index=np.arange(n_games), columns=ids.union(ids_info))
    for game_idx in range(n_games):
        while market.done != ids:
            observations = setting.get_states(ids, market)
            unmatched_agents = [
                agent for agent in agents
                if agent.name not in market.done
            ]
            offers = {
                agent.name: agent.get_offer(observations[agent.name])
                for agent in unmatched_agents
            }
            deals = market.step(offers)
            for agent in unmatched_agents:
                rewards[agent.name][game_idx] = get_reward(agent, deals)[0]
#                 rewards[agent.name+"_deal"][game_idx] = get_reward(agent, deals)[1]
#                 rewards[agent.name+"_resev"][game_idx] = get_reward(agent, deals)[2]
        market.reset()
    return rewards.reindex(sorted(rewards.columns), axis=1)

In [18]:
df_small=play_games(fixed_agents + [rl_agent], setting, 10)

In [19]:
df_small.describe()

,CHAM1,CHAM1-PV,CHAM2,CHAM2-PV,CHAM3,CHAM3-PV,CHAM4,CHAM4-PV,CHAM5,CHAM5-PV
count,10.0,10.000000,10.000000,10.000000,10.0,10.000000,10.000000,10.000000,10.000000,10.0
mean,0.0,0.200000,0.200000,0.200000,0.0,0.200000,0.100000,0.200000,0.300000,0.0
std,0.0,0.421637,0.421637,0.421637,0.0,0.421637,0.316228,0.421637,0.483046,0.0
min,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
25%,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
50%,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
75%,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.750000,0.0
max,0.0,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,0.0


In [20]:
df_big=play_games(fixed_agents + [rl_agent], setting, 500)

In [21]:
df_big.describe()

,CHAM1,CHAM1-PV,CHAM2,CHAM2-PV,CHAM3,CHAM3-PV,CHAM4,CHAM4-PV,CHAM5,CHAM5-PV
count,500.00000,500.000000,500.00000,500.000000,500.000000,500.00000,500.000000,500.000000,500.000000,500.0
mean,0.14400,0.222000,0.13400,0.252000,0.126000,0.24600,0.140000,0.234000,0.260000,0.0
std,0.39443,0.416007,0.38515,0.434596,0.387846,0.43111,0.405869,0.428499,0.439074,0.0
min,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,-1.000000,0.000000,0.0
25%,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0
50%,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0
75%,0.00000,0.000000,0.00000,1.000000,0.000000,0.00000,0.000000,0.000000,1.000000,0.0
max,2.00000,1.000000,2.00000,1.000000,2.000000,1.00000,2.000000,1.000000,1.000000,0.0


In [12]:
# fixed_agents = [
#     UniformRandomAgent('seller', 3, name='CHAM1-PV'),
#     UniformRandomAgent('seller', 3, name='CHAM2-PV'),
#     UniformRandomAgent('seller', 3, name='CHAM4-PV'),
#     UniformRandomAgent('seller', 3, name='CHAM5-PV'),
#     UniformRandomAgent('buyer', 5, name='CHAM1'),
#     UniformRandomAgent('buyer', 5, name='CHAM2'),
#     UniformRandomAgent('buyer', 5, name='CHAM3'),
#     UniformRandomAgent('buyer', 5, name='CHAM4'),
# ]

# rl_seller = GymRLAgent('seller', 3, model=model,name='CHAM3-PV')

In [13]:
# play_games(fixed_agents + [rl_seller], setting, 100).describe()

In [14]:
# play_games(fixed_agents + [rl_seller], setting, 1000).describe()